In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mertbayraktar","key":"b12bfc42522d7f7a19ac34dbc140cce9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d mertbayraktar/belka-encoded


!kaggle datasets download -d ahmedelfazouan/belka-enc-dataset

Dataset URL: https://www.kaggle.com/datasets/ahmedelfazouan/belka-enc-dataset
License(s): unknown
100% 2.86G/2.87G [00:21<00:00, 200MB/s]
100% 2.87G/2.87G [00:21<00:00, 141MB/s]


In [ ]:
!unzip \*.zip -d ./data/

Archive:  belka-enc-dataset.zip
  inflating: ./data/test_enc.parquet  
  inflating: ./data/train_enc.parquet  


In [ ]:
!ls ./data/

test_enc.parquet  train_enc.parquet


In [ ]:
!pip install fastparquet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00


In [ ]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [ ]:
train = pd.read_parquet('/content/data/train_enc.parquet')
test = pd.read_parquet('/content/data/test_enc.parquet')
tst = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/test.parquet')

In [ ]:
pd.options.display.max_rows = 350
pd.options.display.max_columns = 350
train.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,enc37,enc38,enc39,enc40,enc41,enc42,enc43,enc44,enc45,enc46,enc47,enc48,enc49,enc50,enc51,enc52,enc53,enc54,enc55,enc56,enc57,enc58,enc59,enc60,enc61,enc62,enc63,enc64,enc65,enc66,enc67,enc68,enc69,enc70,enc71,enc72,enc73,enc74,enc75,enc76,enc77,enc78,enc79,enc80,enc81,enc82,enc83,enc84,enc85,enc86,enc87,enc88,enc89,enc90,enc91,enc92,enc93,enc94,enc95,enc96,enc97,enc98,enc99,enc100,enc101,enc102,enc103,enc104,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
0,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,4,8,8,8,33,4,12,4,12,12,12,35,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,12,17,31,9,19,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,17,28,8,8,22,8,19,12,12,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,33,8,17,26,28,19,8,17,26,8,19,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,17,26,28,19,33,8,8,26,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train['bind1'].value_counts()

bind1
0    97958646
1      456964
Name: count, dtype: int64

In [ ]:
tst.shape, test.shape, train.shape

((1674896, 6), (1674896, 142), (98415610, 145))

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98415610 entries, 0 to 98415609
Columns: 145 entries, enc0 to bind3
dtypes: int64(3), uint8(142)
memory usage: 15.2 GB


In [ ]:
# # Memory usage of the entire DataFrame
# print(train.memory_usage(deep=True))

# # Total memory usage of the DataFrame
# print(f'Total memory usage: {train.memory_usage(deep=True).sum()} bytes')

In [ ]:
class CFG:

    PREPROCESS = False
    EPOCHS = 20
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05

    NBR_FOLDS = 15
    SELECTED_FOLDS = [0, 2, 4, 6]

    SEED = 2024

In [ ]:
import tensorflow as tf
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)


set_seeds(seed=CFG.SEED)
tf.config.experimental.enable_op_determinism()

In [ ]:
train.shape

(98415610, 145)

In [ ]:
train.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,enc37,enc38,enc39,enc40,enc41,enc42,enc43,enc44,enc45,enc46,enc47,enc48,enc49,enc50,enc51,enc52,enc53,enc54,enc55,enc56,enc57,enc58,enc59,enc60,enc61,enc62,enc63,enc64,enc65,enc66,enc67,enc68,enc69,enc70,enc71,enc72,enc73,enc74,enc75,enc76,enc77,enc78,enc79,enc80,enc81,enc82,enc83,enc84,enc85,enc86,enc87,enc88,enc89,enc90,enc91,enc92,enc93,enc94,enc95,enc96,enc97,enc98,enc99,enc100,enc101,enc102,enc103,enc104,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
0,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,4,8,8,8,33,4,12,4,12,12,12,35,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,12,17,31,9,19,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,17,28,8,8,22,8,19,12,12,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,33,8,17,26,28,19,8,17,26,8,19,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,17,26,28,19,33,8,8,26,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Detect hardware and set up appropriate distribution strategy
try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.TPUStrategy(tpu_resolver)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except Exception as e:
    print("Not on TPU, trying GPU")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        strategy = tf.distribute.MirroredStrategy()
        print("Running on GPU")
        print("Number of GPUs Available: ", len(gpus))
    else:
        strategy = tf.distribute.get_strategy()
        print("Running on CPU")

Running on TPU
REPLICAS:  8


In [ ]:
# train = train.iloc[:1000000, :]

In [ ]:
train.shape

(98415610, 145)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98415610 entries, 0 to 98415609
Columns: 145 entries, enc0 to bind3
dtypes: int64(3), uint8(142)
memory usage: 15.2 GB


In [ ]:
# from tensorflow.keras import mixed_precision

# # Enable mixed precision training
# mixed_precision.set_global_policy('mixed_float16')

## TRANSFORMER

In [ ]:
# # Transformer Block
# class transformer_block(tf.keras.layers.Layer):
#     def __init__(self, dim, num_heads, feed_forward_dim, rate=0.1):
#         super().__init__()
#         self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=dim//num_heads)
#         self.ffn = tf.keras.Sequential(
#             [
#                 tf.keras.layers.Dense(feed_forward_dim, activation="relu"),
#                 tf.keras.layers.Dense(dim),
#             ]
#         )
#         self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6) # 1e-6
#         self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6) # 1e-6
#         self.dropout1 = tf.keras.layers.Dropout(rate)
#         self.dropout2 = tf.keras.layers.Dropout(rate)
#         self.supports_masking = True

#     def call(self, inputs, training, mask):
#         att_mask = tf.expand_dims(mask, axis=-1)
#         att_mask = tf.repeat(att_mask, repeats=tf.shape(att_mask)[1], axis=-1)

#         attn_output = self.att(inputs, inputs, attention_mask = att_mask)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)

# # Positional Encoding Layer
# class positional_encoding_layer(tf.keras.layers.Layer):
#     def __init__(self, num_vocab=5, maxlen=500, hidden_dim=384):
#         super().__init__()
#         self.hidden_dim = hidden_dim
#         self.pos_emb = self.positional_encoding(maxlen, hidden_dim)
#         self.supports_masking = True

#     def call(self, x):
#         maxlen = tf.shape(x)[1]
#         pos_emb = self.pos_emb[:maxlen, :]
#         pos_emb = tf.expand_dims(pos_emb, axis=0)
#         pos_emb = tf.broadcast_to(pos_emb, tf.shape(x))
#         x = tf.math.multiply(x, tf.math.sqrt(tf.cast(self.hidden_dim, tf.float32)))
#         return x + pos_emb

#     def positional_encoding(self, maxlen, hidden_dim):
#         depth = hidden_dim // 2
#         positions = tf.range(maxlen, dtype=tf.float32)[..., tf.newaxis]
#         depths = tf.range(depth, dtype=tf.float32)[np.newaxis, :] / depth
#         angle_rates = 1 / tf.math.pow(10000.0, depths)
#         angle_rads = positions * angle_rates
#         pos_encoding = tf.concat(
#             [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
#             axis=-1
#         )
#         return pos_encoding

# # Define the model function
# def my_model():
#     with strategy.scope():
#         INP_LEN = 142
#         hidden_dim = 128
#         num_vocab = 36

#         inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
#         x = tf.keras.layers.Embedding(input_dim=num_vocab, output_dim=hidden_dim, input_length=INP_LEN, mask_zero=True)(inputs)
#         x = positional_encoding_layer(num_vocab=num_vocab, maxlen=INP_LEN, hidden_dim=hidden_dim)(x)

#         # Add multiple transformer blocks
#         for _ in range(4):
#             x = transformer_block(hidden_dim, num_heads=8, feed_forward_dim=hidden_dim*4)(x)

#         x = tf.keras.layers.GlobalAveragePooling1D()(x)

#         # # Fully connected layers
#         # x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         # x = tf.keras.layers.Dropout(0.1)(x)
#         # x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         # x = tf.keras.layers.Dropout(0.1)(x)
#         # x = tf.keras.layers.Dense(512, activation='relu')(x)
#         # x = tf.keras.layers.Dropout(0.1)(x)

#         x = tf.keras.layers.Dropout(0.5)(x)
#         outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)
#         # outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

#         model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
#         optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay=CFG.WD)
#         loss = 'binary_crossentropy'
#         weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name='avg_precision')]
#         model.compile(
#             loss=loss,
#             optimizer=optimizer,
#             weighted_metrics=weighted_metrics,
#             steps_per_execution=100
#         )
#         return model

## LSTM

In [ ]:

# def my_model():
#     with strategy.scope():
#         INP_LEN = 142
#         NUM_FILTERS = 32
#         hidden_dim = 128

#         inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
#         x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero=True)(inputs)

#         # Three LSTM layers
#         x = tf.keras.layers.LSTM(128, return_sequences=True)(x)
#         x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
#         x = tf.keras.layers.LSTM(32)(x)

#         x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)
#         x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)
#         x = tf.keras.layers.Dense(512, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)

#         outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

#         model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
#         optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay=CFG.WD)
#         loss = 'binary_crossentropy'
#         weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name='avg_precision')]
#         model.compile(
#             loss=loss,
#             optimizer=optimizer,
#             weighted_metrics=weighted_metrics,
#         )
#         return model


## 1D CNN

In [ ]:
def my_model():
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 32
        hidden_dim = 128

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*4, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*5, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)


        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

In [ ]:
from sklearn.utils import class_weight

In [ ]:
train.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,enc37,enc38,enc39,enc40,enc41,enc42,enc43,enc44,enc45,enc46,enc47,enc48,enc49,enc50,enc51,enc52,enc53,enc54,enc55,enc56,enc57,enc58,enc59,enc60,enc61,enc62,enc63,enc64,enc65,enc66,enc67,enc68,enc69,enc70,enc71,enc72,enc73,enc74,enc75,enc76,enc77,enc78,enc79,enc80,enc81,enc82,enc83,enc84,enc85,enc86,enc87,enc88,enc89,enc90,enc91,enc92,enc93,enc94,enc95,enc96,enc97,enc98,enc99,enc100,enc101,enc102,enc103,enc104,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
0,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,4,8,8,8,33,4,12,4,12,12,12,35,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,12,17,31,9,19,35,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,12,4,12,12,12,17,28,8,8,22,8,19,12,12,4,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,33,8,17,26,28,19,8,17,26,8,19,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,22,8,8,28,12,27,12,12,12,17,8,33,12,18,35,12,17,33,8,8,17,26,28,19,33,8,8,26,8,19,35,12,17,33,29,8,3,3,5,32,17,8,8,22,8,19,8,8,17,26,28,19,33,29,30,2,32,19,35,18,19,12,12,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
%%time

FEATURES = [col for col in train.columns if col not in ['bind1', 'bind2', 'bind3']]
TARGETS = ['bind1', 'bind2', 'bind3']

skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42)

all_preds = []
for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):

    if fold not in CFG.SELECTED_FOLDS:
        continue;

    X_train = train.loc[train_idx, FEATURES]
    y_train = train.loc[train_idx, TARGETS]
    X_val = train.loc[valid_idx, FEATURES]
    y_val = train.loc[valid_idx, TARGETS]

    es = tf.keras.callbacks.EarlyStopping(patience=3, monitor="val_loss", mode='min', verbose=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
                                                        save_best_only=True, save_weights_only=True,
                                                    mode='min')
    reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)
    model = my_model()
    history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=CFG.EPOCHS,
            callbacks=[checkpoint, reduce_lr_loss, es],
            batch_size=CFG.BATCH_SIZE,
            verbose=1,
        )
    model.load_weights(f"model-{fold}.h5")
    oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
    print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'macro'))

    preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)

    all_preds.append(preds)

preds = np.mean(all_preds, 0)
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)

## 0.34444842029807915

Epoch 1/20
22426/22426 [==============================] - 519s 22ms/step - loss: 0.0147 - avg_precision: 0.5523 - val_loss: 0.0130 - val_avg_precision: 0.6189 - lr: 0.0010
Epoch 2/20
22426/22426 [==============================] - 494s 22ms/step - loss: 0.0128 - avg_precision: 0.6257 - val_loss: 0.0124 - val_avg_precision: 0.6435 - lr: 0.0010
Epoch 3/20
22426/22426 [==============================] - 494s 22ms/step - loss: 0.0125 - avg_precision: 0.6369 - val_loss: 0.0124 - val_avg_precision: 0.6428 - lr: 0.0010
Epoch 4/20
22426/22426 [==============================] - 494s 22ms/step - loss: 0.0124 - avg_precision: 0.6423 - val_loss: 0.0122 - val_avg_precision: 0.6521 - lr: 0.0010
Epoch 5/20
22426/22426 [==============================] - 494s 22ms/step - loss: 0.0123 - avg_precision: 0.6457 - val_loss: 0.0121 - val_avg_precision: 0.6538 - lr: 0.0010
Epoch 6/20
22426/22426 [==============================] - 495s 22ms/step - loss: 0.0122 - avg_precision: 0.6483 - val_loss: 0.0120 - val_avg

In [ ]:
## tst sırası ile bb3 sırası eşleşmiyor

In [ ]:
# def my_model():
#     with strategy.scope():
#         INP_LEN = 142
#         NUM_FILTERS = 32
#         hidden_dim = 128

#         inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
#         x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero=True)(inputs)

#         # Convolutional layers
#         x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3, activation='relu', padding='valid', strides=1)(x)
#         x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3, activation='relu', padding='valid', strides=1)(x)

#         # LSTM layers
#         x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
#         x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)

#         # Fully connected layers
#         x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)
#         x = tf.keras.layers.Dense(1024, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)
#         x = tf.keras.layers.Dense(512, activation='relu')(x)
#         x = tf.keras.layers.Dropout(0.1)(x)

#         outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

#         model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
#         optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay=CFG.WD)
#         loss = 'binary_crossentropy'
#         weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name='avg_precision')]
#         model.compile(
#             loss=loss,
#             optimizer=optimizer,
#             weighted_metrics=weighted_metrics,
#         )
#         return model

In [ ]:
# %%time
# FEATURES = [f'enc{i}' for i in range(142)]
# TARGETS = ['bind1', 'bind2', 'bind3']
# skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42)

# all_preds = []
# for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):

#     if fold not in CFG.SELECTED_FOLDS:
#         continue;

#     # X_train = train.loc[train_idx, FEATURES]
#     # y_train = train.loc[train_idx, TARGETS]
#     # X_val = train.loc[valid_idx, FEATURES]
#     # y_val = train.loc[valid_idx, TARGETS]

#     # es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
#     # checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
#     #                                                     save_best_only=True, save_weights_only=True,
#     #                                                 mode='min')
#     # reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
#     model = my_model()
#     # history = model.fit(
#     #         X_train, y_train,
#     #         validation_data=(X_val, y_val),
#     #         epochs=CFG.EPOCHS,
#     #         callbacks=[checkpoint, reduce_lr_loss, es],
#     #         batch_size=CFG.BATCH_SIZE,
#     #         verbose=1,
#     #     )
#     model.load_weights(f"model-{fold}.h5")
#     # oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
#     # print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))

#     preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
#     all_preds.append(preds)

# preds = np.mean(all_preds, 0)


In [ ]:
# tst = pd.read_parquet('/content/test.parquet')
# tst['binds'] = 0
# tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
# tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
# tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
# tst[['id', 'binds']].to_csv('submission.csv', index = False)